# 4. Basic file management



## 4.1 Merging files

+ Problem: Is donor age correlated with recipient age in deceased-donor transplants?

Let's first hold the URL to our dataset in a macro:

```stata
global url "https://github.com/jhustata/basic/raw/main/"
```

Now let's import [transplants.dta](https://github.com/jhustata/basic/raw/main/transplants.dta) and 

```stata
use "${url}transplants", clear
lookfor age
```

We only have the recipients age. So we'll have to look elsewhere for the [donor](https://github.com/jhustata/basic/raw/main/donors.dta) age:

```stata
use "${url}donors", clear
lookfor age
```

```stata
. use donors,	clear

. lookfor age

Variable	Storage	Display	Value
name	type	format	label	Variable	label
						
age_don	float	%9.0g		     

. 

```

`donors.dta` has donor age, but not recipient age.

```stata
use donors_recipients, clear
list in 1/10
```

```stata
. use donors_recipients, clear

. list in 1/10

     +--------------------+
     | fake_id   fake_d~d |
     |--------------------|
  1. |       1       4999 |
  2. |       2       4998 |
  3. |       3       4997 |
  4. |       4       4996 |
  5. |       5       4995 |
     |--------------------|
  6. |       6       4994 |
  7. |       7       4993 |
  8. |       8       4992 |
  9. |       9       4991 |
 10. |      10       4990 |
     +--------------------+

```

`donors_recipients.dta` has neither age, but it has fake_id (corresponding to recipients) and fake_don_id (corresponding to donors). Solution?

```stata
use transplants, clear merge 1:1 fake_id ///
    using donors_recipients
```

```stata
. use transplants, clear 

. merge 1:1 fake_id ///
>     using donors_recipients

    Result                      Number of obs
    -----------------------------------------
    Not matched                         3,000
        from master                         0  (_merge==1)
        from using                      3,000  (_merge==2)

    Matched                             2,000  (_merge==3)
    -----------------------------------------
```

+ We expect each fake_id to appear only once in each dataset ("one-to-one merge")
+ fake_id is the variable that appears in both datasets, letting us link them
+ donors_recipients is the dataset that we're merging with the dataset in memory

+ Zero records appeared only in the "master" dataset (in memory before the merge command)
+ 3000 records appeared only in the "using" dataset (donors_recipients.dta)
+ 2000 records appeared in both datasets
+ Stata created a variable called _merge telling us whether each record was master-only, using-only, or both (matched)

```stata
tab _merge 
```

Let's try some fancier syntax:

```stata
use transplants, clear 
merge 1:1 fake_id using donors_recipients, keep(match)
```

Only records that appear in both datasets will remain in memory


```stata
use transplants, clear 
merge 1:1 fake_id using donors_recipients, keep(master match)
```

Only records that appear in the master dataset only, or in both datasets, will remain in memory


```stata
use transplants, clear 
merge 1:1 fake_id using donors_recipients,   gen(mergevar)
```

Instead of creating a variable called _merge, call it mergevar

```stata
use transplants, clear 
merge 1:1 fake_id using donors_recipients,   nogen
```

Don't create any new variables, **NOTE:** if the `_merge` variable already exists, the _merge command will give an error unless you use gen() or nogen

Two merges in a row: 

```stata
use transplants, clear merge 1:1 fake_id ///
    using donors_recipients, ///
    keep(match) nogen
merge m:1 fake_don_id ///
 using donors, keep(match) nogen ///
    keepusing(age_don)
```

Don't load all variables from the new (using) dataset. Just load age_don

```stata
corr age*
```

In our dataset, fake recipient age and fake donor age *** *** correlated.


Merging protip: using merge, keep(match) might drop more records than you expect. If you think all
records will match, it's a good idea to check this assumption

```stata
use transplants, clear merge 1:1 fake_id using donors_recipients, ///
     keep(master match)
assert _merge==3
```

Maybe you don't expect a perfect match, but you want to make sure nearly all of your records match use transplants, clear

```stata
merge 1:1 fake_id using donors_recipients, keep(master match)
quietly count if _merge == 3
assert r(N)/_N > 0.99 //99% of records have _merge==3
```

That's enough in way of an introduction to the `merge` command. It will come in handy only in specific projects that have relevant variables in different datasets.